<a href="https://colab.research.google.com/github/Luke-zm/coursera_learning/blob/main/udemy_pyomo/pyomo_tut3_lp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
# For colab, install the necessary kits
!pip install pyomo
!apt-get install -y -qq glpk-utils

Powerco has 3 electric power plants that supply the needs of 4 cities. Each power plant can supply the following numbers of kWh of electricity.   
Plant 1 -- 35 million;  
Plant 2 -- 50 million;  
Plant 3 -- 40 million;  
The peak power demands in these cities are:
City 1 -- 45 million;  
City 2 -- 20 million;  
City 3 -- 30 million;  
City 4 -- 30 million;  
The cost of sending 1 million kWh of electricity from plant to city can be found below.  Formulate LP to minimize the cost of meeting each city's peak power demand.  
|      | TO |   |   |   | Supply  |  
|------|----|---|---|---|---|  
|   From|  city 1  |  city 2 | city 3  | city 4  |(Million kWh)|  
|Plant 1|  $8      |  $6     |  $10    |  $9     |   35        |   
|Plant 2|  $9      |  $12    |  $13    |  $7     |   50        |   
|Plant 3|  $14     |  $9     |  $16    |  $5     |   40        |   
|Demand |  45      |  20     |  30     |  30     |             |   

First, decide on the decision variables for the problems.  
The decision vraibles are the power to each city from each plant.  
There need to be 2 sets of indices. 1 set representing the plant, the other representing the cities.  
Let E be the ammount of energy in million kWh.  
Let C be the cost of sending the energy in dollars.
Let p be the index for power plants.
Let c be the index for cities.
$$  
\\p = \{1, 2, 3\}  
\\c = \{1, 2, 3, 4 \}  
\\obj:~~~~min~sum(E_{p,c} \times C_{c,p})
\\s.t:
\\sum(E_{1,c})\leq35
\\sum(E_{2,c})\leq50
\\sum(E_{3,c})\leq40
\\sum(E_{p,1})\geq45
\\sum(E_{p,2})\geq20
\\sum(E_{p,3})\geq30
\\sum(E_{p,4})\geq30
\\E_{p,c}\geq0
$$  

In [71]:
# Import the tools
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [72]:
from pyomo.core.base import initializer
# Create the model
model = pyo.ConcreteModel()

# Create the sets for indicies
# Index for power plants
model.p = pyo.Set(initialize=['Plant1', 'Plant2', 'Plant3'])
# Index for cities
model.c = pyo.Set(initialize=['City1', 'City2', 'City3', 'City4'])

In [73]:
# Create the parameters, parameters are stuff that should be fixed
# for the duration of the calculation by optimisation

# Define the limit for the ammount of energy each plant can produce, this is a parameter
plant_cap_dict = {'Plant1':35, 'Plant2':50, 'Plant3':40}
model.plant_cap = pyo.Param(model.p, initialize=plant_cap_dict)
plant_cap = model.plant_cap

# Define the ammount of energy each city will require, this is a parameter.
city_demand_dict = {'City1':45, 'City2':20, 'City3':30, 'City4':30}
model.city_demand = pyo.Param(model.c, initialize=city_demand_dict)
city_demand = model.city_demand

# Define the cost of sending energy from plant to city.
# Index needed is both p and c
cost_dict = {
    ('Plant1', 'City1'):8, ('Plant1', 'City2'):6, ('Plant1', 'City3'):10, ('Plant1', 'City4'):9,
    ('Plant2', 'City1'):9, ('Plant2', 'City2'):12, ('Plant2', 'City3'):13, ('Plant2', 'City4'):7,
    ('Plant3', 'City1'):14, ('Plant3', 'City2'):9, ('Plant3', 'City3'):16, ('Plant3', 'City4'):5
}
model.cost = pyo.Param(model.p, model.c, initialize=cost_dict)
cost = model.cost



In [74]:
# Decision variables
model.energy = pyo.Var(model.p, model.c, within=pyo.NonNegativeReals)
energy = model.energy

In [75]:
# Define objective function
def Objective_rule(model):
  return sum(sum(cost[p, c]*energy[p,c] for p in model.p) for c in model.c)

model.Objf = pyo.Objective(rule = Objective_rule, sense = pyo.minimize)

In [76]:
# Constraints
# Across all cities, from 'Plant1'
# def plant1_cap(model, c):
#   return sum(energy['Plant1', c] for c in model.c) <= plant_cap['Plant1']
# model.plant1_const = pyo.Constraint(model.c, rule=plant1_cap)

# # Across all cities, from 'Plant2'
# def plant2_cap(model, c):
#   return sum(energy['Plant2', c] for c in model.c) <= plant_cap['Plant2']
# model.plant2_const = pyo.Constraint(model.c, rule=plant2_cap)

# # Across all cities, from 'Plant3'
# def plant3_cap(model, c):
#   return sum(energy['Plant3', c]for c in model.c) <= plant_cap['Plant3']
# model.plant3_const = pyo.Constraint(model.c, rule=plant3_cap)

# Across all plants, for 'City1'
# def city1_demand(model, p):
#   return sum(energy[p, 'City1'] for p in model.p )>=city_demand['City1']
# model.city1_demand = pyo.Constraint(model.p, rule=city1_demand)

# # Across all plants, for 'City2'
# def city2_demand(model, p):
#   return sum(energy[p, 'City2'] for p in model.p )>=city_demand['City2']
# model.city2_demand = pyo.Constraint(model.p, rule=city2_demand)

# # Across all plants, for 'City3'
# def city3_demand(model, p):
#   return sum(energy[p, 'City3'] for p in model.p )>=city_demand['City3']
# model.city3_demand = pyo.Constraint(model.p, rule=city3_demand)

# # Across all plants, for 'City4'
# def city4_demand(model, p):
#   return sum(energy[p, 'City4'] for p in model.p )>=city_demand['City4']
# model.city4_demand = pyo.Constraint(model.p, rule=city4_demand)


In [77]:
# Use constraint list from pyomo for more compact formulation

# Personal trial
# Define the overall equation for power plant production capacity
def plant_prod_cap(model, plant, c=None):
  return sum(energy[plant, c] for c in model.c) <= plant_cap[plant]
model.plant_prod_cap_con = pyo.ConstraintList()
for plant in model.p:
  model.plant_prod_cap_con.add(
      expr=plant_prod_cap(model,plant)
  )

# Define the overall equation for city demand constraint
def city_demand_const(model, city, p=None):
  return sum(energy[plant, city] for plant in model.p) >= city_demand[city]
model.city_demand_con = pyo.ConstraintList()
for city in model.c:
  model.city_demand_con.add(
      expr=city_demand_const(model,city)
  )

# Note: added p=None/ c=None, to remind myself these are supposed to
# be carried by the modelinteranlly

Personal experiment is a sucess.  
Using pyo.ConstraintList() is a much more compact and professional method.

In [78]:
Solver = SolverFactory('glpk')
results =Solver.solve(model)

print(results)
print('objective function: ', model.Objf())

for plant in model.p:
  for city in model.c:
    print(f"Enenrgy sent from {plant} to {city} is {energy[plant, city]()}")


Problem: 
- Name: unknown
  Lower bound: 1020.0
  Upper bound: 1020.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 12
  Number of nonzeros: 24
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.007344961166381836
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

objective function:  1020.0
Enenrgy sent from Plant1 to City1 is 0.0
Enenrgy sent from Plant1 to City2 is 10.0
Enenrgy sent from Plant1 to City3 is 25.0
Enenrgy sent from Plant1 to City4 is 0.0
Enenrgy sent from Plant2 to City1 is 45.0
Enenrgy sent from Plant2 to City2 is 0.0
Enenrgy sent from Plant2 to City3 is 5.0
Enenrgy sent from Plant2 to City4 is 0.0
Enenrgy sent from Plant3 to City1 is 0.0
Enenrgy sent from Plant3 to City2 is 10.0
Enenrgy sent from Plant3 to City3 is 0.0
Enenrgy sent from Plant3 to City4 is 3